#### Import necessari

In [66]:
from sklearn.linear_model import Lasso, Ridge
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import random
from sklearn.multiclass import OneVsOneClassifier

#### Caricamento Dataset

In [67]:
train1 = pd.read_json('dataset/Dataset20-21.json');
train2 = pd.read_json('dataset/Dataset21-22.json');
trainSet = pd.concat([train1, train2], ignore_index=True)
validationSet = pd.read_json('dataset/Dataset22-23.json');
testSet = pd.read_json('dataset/Dataset23-24.json');
trainSet.loc[trainSet["risultato"] == "X", "risultato"] = 3
trainSet.loc[trainSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "X", "risultato"] = 3

In [68]:
X_train = trainSet.drop("risultato", axis=1)
y_train = trainSet["risultato"]

X_validation = validationSet.drop("risultato", axis=1)
y_validation = validationSet["risultato"]

X_test = testSet.drop("risultato", axis=1)
y_test = testSet["risultato"]

X_train_noName = X_train
X_train_noName = X_train_noName.drop("squadraa", axis=1)
X_train_noName = X_train_noName.drop("squadrab", axis=1)
X_train_noName = X_train_noName.astype(float)
y_train=y_train.astype(float)

X_validation_noName = X_validation
X_validation_noName = X_validation_noName.drop("squadraa", axis=1)
X_validation_noName = X_validation_noName.drop("squadrab", axis=1)
X_validation_noName = X_validation_noName.astype(float)
y_validation=y_validation.astype(float)

X_test_noName = X_test
X_test_noName = X_test_noName.drop("squadraa", axis=1)
X_test_noName = X_test_noName.drop("squadrab", axis=1)
X_test_noName = X_test_noName.astype(float)

#### Ridge Regression

In [69]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_noName, y_train)

feature_coefficients = ridge_model.coef_
abs_feature_coefficients = abs(feature_coefficients)
selected_features = X_train_noName.columns[feature_coefficients != 0]

In [70]:
feature_importance_df = pd.DataFrame({'Feature': X_train_noName.columns, 'Coefficient': abs_feature_coefficients})
feature_importance_df = feature_importance_df.sort_values(by='Coefficient', ascending=False)

print("Feature Importance:")
print(feature_importance_df)

Feature Importance:
                Feature  Coefficient
6       avversarioa_xag     0.373977
5        avversarioa_xg     0.319154
2        squadraa_xgxag     0.306981
13       squadrab_npzxg     0.290271
18    avversariob_npzxg     0.288067
0           squadraa_xg     0.262318
16      avversariob_xag     0.252846
14     squadrab_npxgxag     0.155243
9   avversarioa_npxgxag     0.150656
4      squadraa_npxgxag     0.130511
8     avversarioa_npzxg     0.121233
1          squadraa_xag     0.121066
12       squadrab_xgxag     0.072652
15       avversariob_xg     0.052531
17    avversariob_xgxag     0.050819
7     avversarioa_xgxag     0.047330
3        squadraa_npzxg     0.028461
11         squadrab_xag     0.023498
10          squadrab_xg     0.014029
19  avversariob_npxgxag     0.013296


In [71]:
threshold = 0.1 
selected_features = feature_importance_df[feature_importance_df['Coefficient'] > threshold]['Feature'].tolist()

In [72]:
X_train_selected = X_train_noName[selected_features]
X_validation_selected = X_validation_noName[selected_features]
X_test_selected = X_test_noName[selected_features]

In [73]:
def evaluate_model(C, kernel, X_train, y_train, X_validation, y_validation):
    svm_model = SVC(C=C, kernel=kernel)
    svm_model = OneVsOneClassifier(svm_model)
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_validation)
    accuracy = accuracy_score(y_validation, y_pred)
    return accuracy

best_C = 1.0
best_kernel = 'linear'
best_accuracy = 0.0
max_iterations = 2000

mask = ~np.isnan(y_validation)
y_validation = y_validation[mask]
X_validation_selected = X_validation_selected[mask] #Rimozione righe con NaN Values


for _ in range(max_iterations):
    new_C = random.uniform(0.1, 10.0)
    new_kernel = random.choice(['linear', 'rbf', 'poly', 'sigmoid'])
    accuracy = evaluate_model(new_C, new_kernel, X_train_selected, y_train, X_validation_selected, y_validation)
    if accuracy > best_accuracy:
        best_C = new_C
        best_kernel = new_kernel
        best_accuracy = accuracy

final_model = SVC(C=best_C, kernel=best_kernel)
final_model.fit(X_train_selected, y_train)
y_pred = final_model.predict(X_validation_selected)
accuracy = accuracy_score(y_validation, y_pred)
y_test_pred = final_model.predict(X_test_selected)
print("Best Hyperparameters: C =", best_C, ", Kernel =", best_kernel, ", Accuracy= ", accuracy)

Best Hyperparameters: C = 3.127464736420431 , Kernel = rbf , Accuracy=  0.5356200527704486
